In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('sentimentdataset.csv')

In [6]:
df.isnull().any()

Unnamed: 0.1    False
Unnamed: 0      False
Text            False
Sentiment       False
Timestamp       False
User            False
Platform        False
Hashtags        False
Retweets        False
Likes           False
Country         False
Year            False
Month           False
Day             False
Hour            False
dtype: bool

In [7]:
df.info

<bound method DataFrame.info of      Unnamed: 0.1  Unnamed: 0  \
0               0           0   
1               1           1   
2               2           2   
3               3           3   
4               4           4   
..            ...         ...   
727           728         732   
728           729         733   
729           730         734   
730           731         735   
731           732         736   

                                                  Text    Sentiment  \
0     Enjoying a beautiful day at the park!        ...   Positive     
1     Traffic was terrible this morning.           ...   Negative     
2     Just finished an amazing workout! 💪          ...   Positive     
3     Excited about the upcoming weekend getaway!  ...   Positive     
4     Trying out a new recipe for dinner tonight.  ...   Neutral      
..                                                 ...          ...   
727  Collaborating on a science project that receiv...       Happy    
728

In [8]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
727    False
728    False
729    False
730    False
731    False
Length: 732, dtype: bool

In [9]:
df.dtypes

Unnamed: 0.1      int64
Unnamed: 0        int64
Text             object
Sentiment        object
Timestamp        object
User             object
Platform         object
Hashtags         object
Retweets        float64
Likes           float64
Country          object
Year              int64
Month             int64
Day               int64
Hour              int64
dtype: object

In [10]:
df.describe

<bound method NDFrame.describe of      Unnamed: 0.1  Unnamed: 0  \
0               0           0   
1               1           1   
2               2           2   
3               3           3   
4               4           4   
..            ...         ...   
727           728         732   
728           729         733   
729           730         734   
730           731         735   
731           732         736   

                                                  Text    Sentiment  \
0     Enjoying a beautiful day at the park!        ...   Positive     
1     Traffic was terrible this morning.           ...   Negative     
2     Just finished an amazing workout! 💪          ...   Positive     
3     Excited about the upcoming weekend getaway!  ...   Positive     
4     Trying out a new recipe for dinner tonight.  ...   Neutral      
..                                                 ...          ...   
727  Collaborating on a science project that receiv...       Happy    
7

In [11]:
df['Text'] = df['Text'].str.strip()
df['Sentiment'] = df['Sentiment'].str.strip()
df['User'] = df['User'].str.strip()
df['Platform'] = df['Platform'].str.strip()
df['Hashtags'] = df['Hashtags'].str.strip()

In [12]:
if 'Unnamed: 0.1' in df.columns and 'Unnamed: 0' in df.columns:
    df['ID'] = df['Unnamed: 0'] + 1 
    df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
elif 'Unnamed: 0.1' in df.columns:
    df['ID'] = df['Unnamed: 0.1'] + 1
    df = df.drop(['Unnamed: 0.1'], axis=1)
elif 'Unnamed: 0' in df.columns:
    df['ID'] = df['Unnamed: 0'] + 1
    df = df.drop(['Unnamed: 0'], axis=1)
else:
    df['ID'] = range(1, len(df) + 1)

cols = df.columns.tolist()
cols.remove('ID')
cols = ['ID'] + cols
df = df[cols]
df = df.set_index('ID')


In [13]:
df.head(5)

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
ID,,,,,,,,,,,,,
1,Enjoying a beautiful day at the park!,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
2,Traffic was terrible this morning.,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
3,Just finished an amazing workout! 💪,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
4,Excited about the upcoming weekend getaway!,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
5,Trying out a new recipe for dinner tonight.,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [14]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hour'] = df['Timestamp'].dt.hour

def categorize_time_period(hour):
    if 5 <= hour < 11:
        return 'Morning'  
    elif 11 <= hour < 15:
        return 'Afternoon'  
    elif 15 <= hour < 19:
        return 'Evening' 
    else:
        return 'Night' 

df['Timestamp'] = df['Hour'].apply(categorize_time_period)

In [15]:
df.head(5)

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
ID,,,,,,,,,,,,,
1,Enjoying a beautiful day at the park!,Positive,Afternoon,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
2,Traffic was terrible this morning.,Negative,Morning,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
3,Just finished an amazing workout! 💪,Positive,Evening,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
4,Excited about the upcoming weekend getaway!,Positive,Evening,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
5,Trying out a new recipe for dinner tonight.,Neutral,Night,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [16]:
def simplify_sentiment(sentiment):
    if pd.isna(sentiment):
        return 'Neutral'
    positive_keywords = [
        'Positive', 'Happiness', 'Joy', 'Love', 'Amusement', 'Enjoyment', 'Admiration', 
        'Affection', 'Awe', 'Acceptance', 'Adoration', 'Elation', 'Euphoria', 'Contentment', 
        'Serenity', 'Gratitude', 'Hope', 'Empowerment', 'Enthusiasm', 'Fulfillment', 
        'Satisfaction', 'Blessed', 'Confidence', 'Accomplishment', 'Optimism', 'Happy',
        'Playful', 'Free-spirited', 'Inspired', 'Confident', 'Thrill', 'Overjoyed',
        'Inspiration', 'Motivation', 'JoyfulReunion', 'Wonderment', 'Enchantment',
        'PlayfulJoy', 'Positivity', 'Kindness', 'Friendship', 'Success', 'Celebration'
    ]
    negative_keywords = [
        'Negative', 'Anger', 'Fear', 'Sadness', 'Disgust', 'Disappointed', 'Shame',
        'Despair', 'Grief', 'Loneliness', 'Jealousy', 'Resentment', 'Frustration',
        'Boredom', 'Anxiety', 'Intimidation', 'Helplessness', 'Envy', 'Regret',
        'Bitterness', 'Yearning', 'Fearful', 'Apprehensive', 'Overwhelmed', 'Jealous',
        'Devastated', 'Frustrated', 'Envious', 'Heartbreak', 'Betrayal', 'Suffering',
        'Isolation', 'Disappointment', 'Sorrow', 'Darkness', 'Desperation', 'Loss',
        'Heartache', 'Bad', 'Hate', 'Sad'
    ]
    sentiment_lower = str(sentiment).lower()
    
    for keyword in positive_keywords:
        if keyword.lower() in sentiment_lower:
            return 'Positive'
    
    for keyword in negative_keywords:
        if keyword.lower() in sentiment_lower:
            return 'Negative'
    
    return 'Neutral'

df['Sentiment_Simplified'] = df['Sentiment'].apply(simplify_sentiment)

In [17]:
df.head(5)

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Sentiment_Simplified
ID,,,,,,,,,,,,,,
1,Enjoying a beautiful day at the park!,Positive,Afternoon,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12,Positive
2,Traffic was terrible this morning.,Negative,Morning,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8,Negative
3,Just finished an amazing workout! 💪,Positive,Evening,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15,Positive
4,Excited about the upcoming weekend getaway!,Positive,Evening,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18,Positive
5,Trying out a new recipe for dinner tonight.,Neutral,Night,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19,Neutral


In [ ]:
df.to_csv('sentimentdata_cleaning.csv', index=False)